# Using this Notebook

In the **User Configuration** section, you can set the file name for the spreadsheet. You can also select the chopping mode. Options are:

*   sentences (default): Attempts to break the text into whole sentences. Tends to produce coherent text but can be confused by unusual characters and line breaks.
*   line: breaks the input by lines without regard to punctuation or content.
*   phrases: breaks the input into phrases of `min_words` to `max_words` in length.

Once you have set the configuration options, run the notebook by selecting "Runtime - Run All" from the notebook menu.

In the **File Upload** section, you will now be able to click the Choose Files button to upload your files. After processing is complete (different options take different amounts of time), the oracle file should download automatically.



# Imports

In [ ]:
from glob import glob
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font, Color, PatternFill
from random import randint
from google.colab import files

# User Configuration

In [ ]:
# {run: "auto"}

output_file_name = "oracle" #@param {type:"string"}
mode = "sentences" #@param ["line", "phrases", "sentences"] {type:"string"}
min_words = 2 #@param {type:"integer"}
max_words = 7 #@param {type:"integer"}

# File Upload

In [ ]:
input_files = files.upload()

# Implementation

In [ ]:
if mode == "phrases":
  match_lines = False
  word_split_token = " "
  characters_to_replace_with_word_split_token = ",!?\n\r\\"

In [ ]:
if mode == "sentences":
  match_lines = False
  word_split_token = "."
  characters_to_replace_with_word_split_token = "\r\n\\"

In [ ]:
if mode == "lines":
  match_lines = True
  word_split_token = "."
  characters_to_replace_with_word_split_token = "\n\r\\"

In [ ]:
min_phrase_length = 16
output_file_name += ".xlsx"

if min_words < 2: 
  min_words = 2
if max_words < min_words:
  max_words = min_words

In [ ]:
def num_words(token, a=min_words, b=max_words):
  if token == ".":
    return 1
  else:
    return randint(a, b) + randint(a, b)

In [ ]:
def clean(phrase: str):
  phrase = phrase.replace("â€", ",")
  for char in characters_to_replace_with_word_split_token:
    phrase = phrase.replace(char, word_split_token)
  return phrase.strip()

In [ ]:
def get_words(input_files:list, word_split_token:str):
  words = []
  for book in input_files.values():
    words.extend(clean(str(book)).split(word_split_token))
  return words

In [ ]:
def get_phrases_random(input_files, word_split_token):
  phrases = []
  words = get_words(input_files, word_split_token)
  while True:
    try:
      n = num_words(word_split_token)
      phrase = [words.pop(0) for _ in range(n)]
    except IndexError:
      phrase = words
      break
    finally:
      phrase = " ".join(phrase).strip()
      if len(phrase) >= min_phrase_length:
        phrases.append(phrase)
  return phrases

In [ ]:
def get_phrases_lines(input_files):
  lines = []
  for book in input_files.values():
    book = str(book)
    for line in book:
      line = clean(line)
      if len(line) >= min_phrase_length:
        lines.append(line)
  return lines

In [ ]:
def get_phrases(input_files, match_lines=False, word_split_token=" "):
  if match_lines:
    return get_phrases_lines(input_files)
  else:
    return get_phrases_random(input_files, word_split_token)

In [ ]:
phrases = get_phrases(input_files, match_lines, word_split_token)

# Create the Output Spreadsheet

In [ ]:
wb = Workbook()

# Setup the UI worksheet
ws = wb.active
ws.title = "Oracle"
ws["A1"] = "Total Phrases:"
ws["B1"] = '=COUNTIF(phrases!$A$1:$A$100000,">0")'

ws["C1"] = """Press F9 (Recalculate) to choose a new set of phrases.

If you type a number into a Keep cell, then that phrase will be kept
when you randomize the phrases. Be careful not to copy the formula or you will
get errors about 'circular references'.

If you have long phrases, you may want to set column C text to wrap."""

ws["A4"] = "Roll"
ws["B4"] = "Keep"
ws["C4"] = "Phrase"

# formula to select the phrases
num_phrases = 10
first_row = 5
last_row = first_row + num_phrases
for row in range(first_row, last_row):
  ws[f"A{row}"] = f"=IF(B{row}>0, B{row}, RANDBETWEEN(1,$B$1))"
  ws[f"C{row}"] = f"=VLOOKUP(A{row},phrases!$A$1:$B$100000,2,1)"

# add some formatting and styles
ws.column_dimensions["C"].width = 100

bold_font = Font(bold=True)
alignment = Alignment(wrap_text=True)

headingRGB = "D8E4BC"
headingFill = PatternFill(
  start_color=headingRGB,
  end_color=headingRGB,
  fill_type='solid')

ws["A1"].alignment = alignment
for cell in ws["C"]: cell.alignment = alignment
ws["A1"].fill = headingFill
ws["B1"].fill = headingFill
ws["C1"].fill = headingFill
ws["A1"].font = bold_font
ws["A4"].font = bold_font
ws["B4"].font = bold_font
ws["C1"].font = bold_font
ws["C4"].font = bold_font

# Write the phrases into a new worksheet
wb.create_sheet(title="phrases")

phrases_sheet = wb["phrases"]
for i, phrase in enumerate(phrases):
    phrases_sheet.append((i, phrase))

# Save and download the workbook
wb.save(output_file_name)
files.download(output_file_name)